**Project name: Decoding The Subjective Ratings of Valence From EEG Activation Evoked by Emotional Visual Stimuli**

**Dataset: [DEAP dataset](https://www.eecs.qmul.ac.uk/mmv/datasets/deap/)**

-------------

**Import libraries, packages, etc.**

In [5]:
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import pickle
import os
import h5py
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
!pip install mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.5 MB 17.4 MB/s 


# **Mounting the data**




**Mount the data from Pelin's Google Drive**

In [6]:
from google.colab import drive
drive.mount('/content/drive')
# drive.mount("/content/drive", force_remount=True)
%cd /content/drive/MyDrive/Education/Course & Content/Neuromatch Academy - Computational Neuroscience/DEAP Project
!ls

Mounted at /content/drive
/content/drive/MyDrive/Education/Course & Content/Neuromatch Academy - Computational Neuroscience/DEAP Project
deap_data_all.mat	    s01.dat  s08.dat  s15.dat  s22.dat	s29.dat
DEAP_project.ipynb	    s02.dat  s09.dat  s16.dat  s23.dat	s30.dat
metadata_csv		    s03.dat  s10.dat  s17.dat  s24.dat	s31.dat
metadata_xls		    s04.dat  s11.dat  s18.dat  s25.dat	s32.dat
NMA-Project-Slides.gslides  s05.dat  s12.dat  s19.dat  s26.dat
project-abstract.pdf	    s06.dat  s13.dat  s20.dat  s27.dat
Results			    s07.dat  s14.dat  s21.dat  s28.dat


**OYKU's Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# drive.mount("/content/drive", force_remount=True)
%cd /content/drive/MyDrive/DEAP Project
!ls

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1S0VoYLABvBxVrdOqso5P6VJtvk1Iat5n/DEAP Project
deap_data_all.mat   s03.dat  s09.dat  s15.dat  s21.dat	s27.dat
DEAP_project.ipynb  s04.dat  s10.dat  s16.dat  s22.dat	s28.dat
metadata_csv	    s05.dat  s11.dat  s17.dat  s23.dat	s29.dat
metadata_xls	    s06.dat  s12.dat  s18.dat  s24.dat	s30.dat
s01.dat		    s07.dat  s13.dat  s19.dat  s25.dat	s31.dat
s02.dat		    s08.dat  s14.dat  s20.dat  s26.dat	s32.dat


**YAGMUR's Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# drive.mount("/content/drive", force_remount=True)
%cd /content/drive/MyDrive/Education/Course & Content/Neuromatch Academy - Computational Neuroscience/DEAP Project
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: '/content/drive/MyDrive/Education/Course & Content/Neuromatch Academy - Computational Neuroscience/DEAP Project'
/content/drive/MyDrive/DEAP_Dataset
drive-download-20220715T133943Z-001  participant_ratings.csv
online_ratings.csv		     participant_ratings.xls
online_ratings.xls		     video_list.csv
participant_questionnaire.csv	     video_list.xls
participant_questionnaire.xls


**ZEYNEP's Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# drive.mount("/content/drive", force_remount=True)
%cd /content/drive/MyDrive/DEAP_Dataset

Mounted at /content/drive
/content/drive/MyDrive/DEAP_Dataset


# **MAIN**

## **Explore the data**

**Check if everything works fine**

In [7]:
df = pd.read_csv('metadata_csv/participant_ratings.csv')
df

,Participant_id,Trial,Experiment_id,Start_time,Valence,Arousal,Dominance,Liking,Familiarity
0,1,1,5,1695918,6.96,3.92,7.19,6.05,4.0
1,1,2,18,2714905,7.23,7.15,6.94,8.01,4.0
2,1,3,4,3586768,4.94,6.01,6.12,8.06,4.0
3,1,4,24,4493800,7.04,7.09,8.01,8.22,4.0
4,1,5,20,5362005,8.26,7.91,7.19,8.13,1.0
...,...,...,...,...,...,...,...,...,...
1275,32,36,1,30916115,8.13,4.83,9.00,4.87,2.0
1276,32,37,18,31703107,8.03,7.06,8.96,8.18,5.0
1277,32,38,3,32482079,8.05,7.09,8.86,7.08,5.0
1278,32,39,25,33248922,4.01,7.17,8.01,6.94,2.0


**Import .dat files into array** 

In [13]:
for i in [1, 3, 4, 5, 6, 7, 8, 9, 10, 11]:
  if i<10:
      f = ['s0' + str(i) + '.dat']
  else:
    f = ['s' + str(i) + '.dat']
  print(f)

['s01.dat']
['s03.dat']
['s04.dat']
['s05.dat']
['s06.dat']
['s07.dat']
['s08.dat']
['s09.dat']
['s10.dat']
['s11.dat']


**Test:** first 10 subjects

In [2]:
fname = "s01.dat","s02.dat", "s03.dat", "s04.dat", "s05.dat", "s06.dat", "s07.dat", "s08.dat", "s09.dat", "s10.dat"
fname

('s01.dat',
 's02.dat',
 's03.dat',
 's04.dat',
 's05.dat',
 's06.dat',
 's07.dat',
 's08.dat',
 's09.dat',
 's10.dat')

In [ ]:
data = np.empty((10,40,40,8064))
data[:] = np.nan
data.shape

(10, 40, 40, 8064)

In [ ]:
for i in np.arange(0, 10):
  print(fname[i])
  x = pickle.load(open(fname[i], "rb"), encoding="latin1")
  data[i] = x["data"]                                    # shape is (40, 40, 8064) i.e., (trials, channels, samples)

s01.dat
s03.dat
s04.dat
s05.dat
s06.dat
s07.dat
s08.dat
s09.dat
s10.dat
s11.dat


In [ ]:
s1_data = data[1,:,:,:] # trial,channel,sample

In [ ]:
f_sampling = 128
time = np.linspace(-3, 60, 63 * f_sampling)
time.shape

In [ ]:
sub_data = s1_data[20, 15,: ]
plt.plot(time,sub_data)

# **CNN PART**

Check where you are, in which directory.

In [8]:
# Print the current working directory
print("Current working directory: {0}".format(os.getcwd()))

Current working directory: /content/drive/MyDrive/Education/Course & Content/Neuromatch Academy - Computational Neuroscience/DEAP Project


In [9]:
f = h5py.File('deap_data_all.mat','r')
eeg = f.get('eeg_all')
eeg = np.array(eeg) # For converting to a NumPy array
val=f.get('valence_all')
val=np.array(val)
np.random.seed(1)

In [10]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(8064, 32, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 8062, 30, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 4031, 15, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 4029, 13, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 2014, 6, 64)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 2012, 4, 64)       36928     
                                                                 
Total params: 55,744
Trainable params: 55,744
Non-traina

In [14]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 8062, 30, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 4031, 15, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 4029, 13, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 2014, 6, 64)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 2012, 4, 64)       36928     
                                                                 
 flatten (Flatten)           (None, 515072)            0

In [16]:
class_names = ['low_val','high_val']
val=np.where(val > 4.5, 1, 0)

In [17]:
ind=np.arange(0,1280,1)
test_ind=np.random.choice(ind,size=256, replace=False)
eeg.shape

(8064, 32, 1280)

In [19]:
eeg_t=np.transpose(eeg,(2,0,1))
eeg_t.shape

val_t=np.transpose(val,(1,0))
val_t.shape

(1280, 1)

In [21]:
test_eeg = eeg_t[test_ind,:,:]
test_eeg.shape

test_val=val_t[test_ind,:]
test_val.shape

(256, 1)

In [22]:
train_ind=np.setdiff1d(ind,test_ind)
train_ind.shape

train_eeg = eeg_t[train_ind,:,:]
train_eeg.shape

train_val = val_t[train_ind,:]
train_val.shape

(1024, 1)

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_eeg, train_val, epochs=10, 
                    validation_data=(test_eeg, test_val))

Epoch 1/10
32/32 [==============================] - 329s 10s/step - loss: 103.6765 - accuracy: 0.5557 - val_loss: 0.6894 - val_accuracy: 0.6172
Epoch 2/10
32/32 [==============================] - 330s 10s/step - loss: 0.6773 - accuracy: 0.6494 - val_loss: 0.6807 - val_accuracy: 0.6172
Epoch 3/10
32/32 [==============================] - 322s 10s/step - loss: 0.6402 - accuracy: 0.6611 - val_loss: 0.6836 - val_accuracy: 0.6055
Epoch 4/10
32/32 [==============================] - ETA: 0s - loss: 0.5693 - accuracy: 0.7158

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_eeg,  test_val, verbose=2)

In [ ]:
print(test_acc)